In [1]:
import numpy as np
import pandas as pd
import os
from tqdm import tqdm
import xarray as xr
import xesmf as xe
import matplotlib.pyplot as plt
base_dir = f'/mnt/lustre/koa/class/atmo449_class/students/team_1_flood_risk/'

# Rainfall data

subhourly -> hourly is done by mean aggregation. Only keep data if all 4 timestamps exist.<br>
hourly -> 3 hourly is done by max aggregation. Keep data if any timestamp exists.

In [2]:
df_station_data = pd.read_csv(f"{base_dir}/raw_data/station_metadata.csv")
available_station_ids = [int(i.split('.')[0]) for i in os.listdir(f"{base_dir}/preprocessed_data/selected_flowgauge_15mins")]

In [3]:
aggregation = 'mean'
temp_res = 1

In [ ]:
for station_id in tqdm(available_station_ids):
    df_obs = pd.read_csv(f"{base_dir}/preprocessed_data/selected_flowgauge_15mins/{station_id}.csv")
    df_obs['hst_timestamp'] = pd.to_datetime(df_obs['hst_timestamp'])
    df_obs = df_obs.set_index('hst_timestamp', drop=True)
    df_obs_hourly_means_all = df_obs.resample('h').mean()
    df_obs_hourly_means = df_obs_hourly_means_all[(df_obs.groupby(pd.Grouper(freq='h')).size() == 4)]

    if temp_res == 1: # in this case aggregation does not matter
        df_obs_hourly_means.to_csv(f"{base_dir}/preprocessed_data/selected_flowgauge_1hourly/{station_id}.csv")
        continue

    # this is ignored if temp_res == 1
    if aggregation == 'mean':
        df_3hourly = df_obs_hourly_means.resample('3h', closed='left').mean()
    else:
        df_3hourly = df_obs_hourly_means.resample('3h', closed='left').max()
    df_3hourly.to_csv(f"{base_dir}/preprocessed_data/selected_flowgauge_3hourly/{aggregation}/{station_id}.csv")

  0%|                                                                                                                                                                                                 | 0/32 [00:00<?, ?it/s]


# ERA5 data

Regrid, resample

In [2]:
ds_gcm = xr.open_dataset(f"{base_dir}/raw_data/GFDL/GFDL_CM4C192_pr_hawaii_2007_2014.nc")
output_grid = xr.Dataset({"lon": ds_gcm.lon.values, "lat": ds_gcm.lat.values})

In [4]:
# get output grid
ds_gcm = xr.open_dataset(f"{base_dir}/raw_data/GFDL/GFDL_CM4C192_pr_hawaii_2007_2014.nc")
output_grid = xr.Dataset({"lon": ds_gcm.lon.values, "lat": ds_gcm.lat.values})

# # load ERA5 data
for year in tqdm(range(2007, 2025)):
    filename = f"{base_dir}/preprocessed_data/hourly_ERA5/ERA5_{year}.nc"
    ds_era5 = xr.open_dataset(filename)#, decode_cf=False)
    input_grid = xr.Dataset({"lat": ds_era5.latitude.values, "lon": ds_era5.longitude.values})
    regridder = xe.Regridder(input_grid, output_grid, 'conservative')
    era5_regridded_3hourly = regridder(ds_era5.rename({'latitude': 'lat', 'longitude': 'lon'})).resample(time="3h").mean()
    era5_regridded_3hourly.to_netcdf(f"{base_dir}/preprocessed_data/regridded_3hourly_ERA5/{year}.nc")

  0%|                                                                                                                                                                                                 | 0/18 [00:00<?, ?it/s]

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 18/18 [01:55<00:00,  6.39s/it]


# Extract closest pixel, concat along years

In [27]:
df_station_data = pd.read_csv(f"{base_dir}/raw_data/station_metadata.csv")
available_station_ids = [int(i.split('.')[0]) for i in os.listdir(f"{base_dir}/preprocessed_data/selected_flowgauge_15mins")]

In [28]:
for i, row in tqdm(df_station_data[df_station_data['station_id'].isin(available_station_ids)].iterrows(), total=len(available_station_ids)):
    all_year_data = []
    # for year in range(2007, 2025):
    for year in range(1974, 2025):
        all_year_data.append(xr.open_dataset(f"{base_dir}/preprocessed_data/regridded_3hourly_ERA5/{year}.nc")['tp'].sel(lat=row.latitude, lon=row.longitude, method='nearest'))
    xr.concat(all_year_data, dim='time').to_netcdf(f"{base_dir}/preprocessed_data/station_ERA5/{row.station_id}.nc")

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 32/32 [00:49<00:00,  1.55s/it]


# Misc ERA5

In [29]:
# create non-gridded 3 hourly ERA5
# # load ERA5 data
for year in tqdm(range(1974, 2025)):
    filename = f"{base_dir}/preprocessed_data/hourly_ERA5/ERA5_{year}.nc"
    ds_era5 = xr.open_dataset(filename)
    era5_3hourly = ds_era5.rename({'latitude': 'lat', 'longitude': 'lon'}).resample(time="3h", offset='1h').mean()
    era5_3hourly.to_netcdf(f"{base_dir}/preprocessed_data/misc/original_grid_3hourly_ERA5/{year}.nc")

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 51/51 [03:54<00:00,  4.60s/it]


In [12]:
df_station_data = pd.read_csv(f"{base_dir}/raw_data/station_metadata.csv")
available_station_ids = [int(i.split('.')[0]) for i in os.listdir(f"{base_dir}/preprocessed_data/selected_flowgauge_15mins")]

In [13]:
for i, row in tqdm(df_station_data[df_station_data['station_id'].isin(available_station_ids)].iterrows(), total=len(available_station_ids)):
    all_year_data = []
    for year in range(2007, 2025):
        all_year_data.append(xr.open_dataset(f"{base_dir}/preprocessed_data/misc/original_grid_3hourly_ERA5/{year}.nc")['tp'].sel(lat=row.latitude, lon=row.longitude, method='nearest'))
    xr.concat(all_year_data, dim='time').to_netcdf(f"{base_dir}/preprocessed_data/misc/original_grid_3hourly_station_ERA5/{row.station_id}.nc")

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 32/32 [00:39<00:00,  1.24s/it]


### Hourly ERA5 on original grid

In [7]:
df_station_data = pd.read_csv(f"{base_dir}/raw_data/station_metadata.csv")
available_station_ids = [int(i.split('.')[0]) for i in os.listdir(f"{base_dir}/preprocessed_data/selected_flowgauge_15mins")]

In [ ]:
for i, row in tqdm(df_station_data[df_station_data['station_id'].isin(available_station_ids)].iterrows(), total=len(available_station_ids)):
    all_year_data = []
    for year in range(2007, 2025):
        all_year_data.append(xr.open_dataset(f"{base_dir}/preprocessed_data/hourly_ERA5/ERA5_{year}.nc")['tp'].rename({'latitude': 'lat', 'longitude': 'lon'}).sel(lat=row.latitude, lon=row.longitude, method='nearest'))
    xr.concat(all_year_data, dim='time').to_netcdf(f"{base_dir}/preprocessed_data/misc/original_grid_1hourly_station_ERA5/{row.station_id}.nc")

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 32/32 [02:41<00:00,  5.05s/it]


# GFDL Data

In [15]:
df_station_data = pd.read_csv(f"{base_dir}/raw_data/station_metadata.csv")
available_station_ids = [int(i.split('.')[0]) for i in os.listdir(f"{base_dir}/preprocessed_data/selected_flowgauge_15mins")]

In [23]:
filename = f"{base_dir}/wasfia/GFDL_CM4C192_pr_hawaii_1981_2010.nc"
ds_gfdl = xr.open_dataset(filename)
for i, row in tqdm(df_station_data[df_station_data['station_id'].isin(available_station_ids)].iterrows(), total=len(available_station_ids)):
    ds_gfdl['pr'].sel(lat=row.latitude, lon=row.longitude, method='nearest').to_netcdf(f"{base_dir}/preprocessed_data/station_cmip_historical/{row.station_id}.nc")

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 32/32 [01:21<00:00,  2.53s/it]
